# Custom Training with TensorFlow in Sagemaker

# Download Data

In [1]:
%matplotlib inline

import os
import tarfile
import urllib
import shutil
import json
import random
import numpy as np
import tensorflow as tf
import sagemaker

from PIL import Image
from matplotlib import pyplot as plt

urls = ['http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz',
        'http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz']

print('Libraries imported')


Libraries imported


In [2]:
def download_and_extract(data_dir, download_dir):
    for url in urls:
        target_file = url.split('/')[-1]
        if target_file not in os.listdir(download_dir):
            print('Downloading', url)
            urllib.request.urlretrieve(url, os.path.join(download_dir, target_file))
            tf = tarfile.open(url.split('/')[-1])
            tf.extractall(data_dir)
        else:
            print('Already downloaded', url)

def get_annotations(file_path, annotations={}):
    
    with open(file_path, 'r') as f:
        rows = f.read().splitlines()

    for i, row in enumerate(rows):
        image_name, _, _, _ = row.split(' ')
        class_name = image_name.split('_')[:-1]
        class_name = '_'.join(class_name)
        image_name = image_name + '.jpg'
        
        annotations[image_name] = 'cat' if class_name[0] != class_name[0].lower() else 'dog'
    
    return annotations

In [3]:
if not os.path.isdir('data'):
    os.mkdir('data')

download_and_extract('data', '.')

# Dataset for Training

In [4]:
annotations = get_annotations('data/annotations/trainval.txt')
annotations = get_annotations('data/annotations/test.txt', annotations)

total_count = len(annotations.keys())
print('Total examples', total_count)

Total examples 7349


In [5]:
next(iter(annotations.items()))

('Abyssinian_100.jpg', 'cat')

In [6]:
classes = ['cat', 'dog']
sets = ['train', 'validation']
root_dir = 'custom_data'

if not os.path.isdir(root_dir):
    os.mkdir(root_dir)
    
for set_name in sets:
    if not os.path.isdir(os.path.join(root_dir, set_name)):
        os.mkdir(os.path.join(root_dir, set_name))
    for class_name in classes:
        folder = os.path.join(root_dir, set_name, class_name)
        if not os.path.isdir(folder):
            os.mkdir(folder)

Copy the files to correct set/ class folders

In [7]:
for image, class_name in annotations.items():
    target_set = 'validation' if random.randint(0, 99) < 20 else 'train'
    target_path = os.path.join(root_dir, target_set, class_name, image)
    shutil.copy(os.path.join('data/images/', image), target_path)

In [8]:
sets_counts = {
    'train': 0,
    'validation': 0
}

for set_name in sets:
    for class_name in classes:
        path = os.path.join(root_dir, set_name, class_name)
        count = len(os.listdir(path))
        print(path, 'has', count, 'images')
        sets_counts[set_name] += count

print(sets_counts)

custom_data/train/cat has 1915 images
custom_data/train/dog has 4012 images
custom_data/validation/cat has 456 images
custom_data/validation/dog has 966 images
{'train': 5927, 'validation': 1422}


# Training Script - Create Model

In [9]:
%%writefile train.py

import tensorflow as tf
import argparse
import os
import json

def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,pooling='avg',weights='imagenet',input_shape=(128,128,3)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1,activation='sigmoid')
        
    ])
    model.layers[0].trainable = False
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics =['accuracy'])
    return model

Writing train.py


# Training Script - Data Generators

In [12]:
%%writefile -a train.py

def create_data_generators(root_dir,batch_size):
    train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function= tf.keras.applications.mobilenet_V2.preprocess_input,
    horizontal_flip=True,zoom_range=[0.8,1.2],rotation_range=20).flow_from_directory(
        os.path.join(root_dir,'validation'),target_size=(128,128),
        batch_size=batch_size,class_mode='binary')
    
    
    val_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function= tf.keras.applications.mobilenet_V2.preprocess_input).flow_from_directory(
        os.path.join(root_dir,'train'),target_size=(128,128),
        batch_size=batch_size,class_mode='binary')
    
    return train_data_generator,val_data_generator 

Appending to train.py


# Training Script - Putting it Together

In [13]:
%%writefile -a train.py

if __name__== '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--epochs',type=int,default=3)
    parser.add_argument('--batch_size',type=int,default=16)
    parser.add_argument('--steps',type=int,default=int(5927/16))
    parser.add_argument('--val_steps',type=int,default=int(1422/16))
    
    parser.add_argument('--model_dir',type=str)
    parser.add_argument('--sm-model-dir',type=str,default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train',type=str,default=os.environ.get('SM_CHANNEL_TRAINING'))
    
    args, _ = parser.parse_known_args()
    
    local_output_dir = args.sm_model_dir
    local_root_dir = args.train
    batch_size = args.batch_size
    
    model = create_model()
    train_gen, val_gen = create_data_generators(local_root_dir,batch_size)
    
    _ = model.fit(train_gen,epochs = args.epochs,steps_per_epoch = args.steps,
                  validation_data = val_gen, validation_steps = args.val_steps )
    
    model.save(os.path.join(local_output_dir,'model','1'))

Appending to train.py


# Upload Dataset to S3

In [21]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket_name = 'petimages1'

print("Uploading..")
s3_data_path = sess.upload_data(path=root_dir,bucket = bucket_name,key_prefix='data')
print('Uploading to ',s3_data_path)

Uploading..
Uploading to  s3://petimages1/data


# Train with TensorFlow Estimator

In [24]:
from sagemaker.tensorflow import TensorFlow

pets_estimator = TensorFlow( entry_point = 'train.py',role=role,train_instance_type = 'ml.p2.xlarge',
                                train_instance_count = 1,framework_version = '2.1.0', py_version = 'py3',
                                output_path = 's3://petimages1/')

In [27]:
pets_estimator.fit(s3_data_path)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-09-21 10:42:11 Starting - Starting the training job...
2020-09-21 10:42:13 Starting - Launching requested ML instances.........
2020-09-21 10:43:53 Starting - Preparing the instances for training......
2020-09-21 10:45:07 Downloading - Downloading input data.........
2020-09-21 10:46:28 Training - Downloading the training image...
2020-09-21 10:46:52 Training - Training image download completed. Training in progress.2020-09-21 10:46:57,476 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-09-21 10:46:57,885 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "model_dir": "s3://petimages1/tensorflow-training-2020-09-21-10-17-

2020-09-21 10:47:32.907877: W tensorflow/python/util/util.cc:319] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /opt/ml/model/model/1/assets
INFO:tensorflow:Assets written to: /opt/ml/model/model/1/assets
#015  1/370 [..............................] - ETA: 33:25 - loss: 0.7324 - accuracy: 0.6250#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  4/370 [..............................] - ETA: 8:23 - loss: 0.9407 - accuracy: 0.5156 #


2020-09-21 10:47:51 Uploading - Uploading generated training model
2020-09-21 10:47:51 Completed - Training job completed
Training seconds: 164
Billable seconds: 164


# Deploy TensorFlow Model

In [29]:
pets_predictor = pets_estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')
print("\nModel Deployed")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
Using already existing model: tensorflow-training-2020-09-21-10-42-11-419


-----------!
Model Deployed


# Final Predictions

In [30]:
cat_dir = 'custom_data/validation/cat/'
cat_images = [os.path.join(cat_dir, x) for x in os.listdir(cat_dir)]
print(cat_images[0])

dog_dir = 'custom_data/validation/dog/'
dog_images = [os.path.join(dog_dir, x) for x in os.listdir(dog_dir)]
print(dog_images[0])

custom_data/validation/cat/Birman_82.jpg
custom_data/validation/dog/german_shorthaired_54.jpg


In [31]:
def get_pred(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    img = np.expand_dims(img, axis=0)

    results = pets_predictor.predict(img)
    return results

In [32]:
image_path = cat_images[0]
results = get_pred(image_path)

print(results)

{'predictions': [[0.0275303908]]}


In [33]:
class_id = int(np.squeeze(results['predictions']) > 0.5)
print('Predicted class_id:', class_id, 'with class_name:', classes[class_id])

Predicted class_id: 0 with class_name: cat


In [34]:
image_path = dog_images[0]
results = get_pred(image_path)

print(results)

{'predictions': [[0.902133286]]}


In [35]:
class_id = int(np.squeeze(results['predictions']) > 0.5)
print('Predicted class_id:', class_id, 'with class_name:', classes[class_id])

Predicted class_id: 1 with class_name: dog


# Delete Model Endpoint

In [36]:
sess.delete_endpoint(pets_predictor.endpoint)